In [104]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [105]:
conv = nn.Conv1d(in_channels=20, out_channels=20, kernel_size=3)

In [122]:
x = torch.randn(20,4)

In [124]:
print(x)

tensor([[ 1.4633, -0.0547,  1.3137, -1.3222],
        [-1.4318, -0.2267,  0.4703, -0.0855],
        [ 1.5460,  2.4278, -1.0971,  0.7958],
        [-1.2056,  0.1761,  0.5204,  2.1128],
        [-0.0873,  1.6430,  0.0553, -0.6600],
        [-0.4570, -0.2419,  0.9161, -0.8416],
        [-0.3707,  0.3603,  1.7400, -2.0694],
        [-0.9121, -1.6947,  1.9736, -1.0083],
        [-1.6632,  0.4293, -1.1267, -0.8588],
        [-1.3795,  2.2254, -0.7376,  1.0695],
        [ 0.3118,  0.3696, -0.6152,  0.0399],
        [ 0.4593,  0.0976,  0.3786,  1.4477],
        [ 1.9312, -0.7486, -1.2858, -0.3124],
        [-1.1709,  0.9252,  0.2368,  1.7426],
        [-0.4306, -0.8855,  1.0167,  1.2084],
        [ 0.3626,  0.1810,  1.4095,  1.4063],
        [ 0.4529,  0.3516,  0.6279, -0.9043],
        [-0.7568, -0.5044,  1.5473, -0.9308],
        [-0.1760, -1.3123,  0.7106, -0.8567],
        [-1.2217, -0.6242, -1.2205,  2.0975]])


In [126]:
x = conv(x)

In [127]:
print(x)

tensor([[ 0.3437, -0.1859],
        [ 0.0309,  0.7900],
        [ 0.0541,  0.1204],
        [-1.0698, -0.8762],
        [-0.7035,  0.1976],
        [-0.3844,  1.1223],
        [-0.5285,  1.6792],
        [-0.4279, -0.2284],
        [-0.7689,  0.2311],
        [ 1.0001, -0.0284],
        [-0.2504, -0.2195],
        [-0.1710,  0.0692],
        [ 0.2937, -1.0192],
        [-0.7900,  1.1797],
        [ 0.6989, -0.1807],
        [-0.5894,  0.2693],
        [ 0.0839, -0.4948],
        [ 0.4644, -0.7051],
        [ 0.0099,  0.1522],
        [-0.2532,  0.5403]], grad_fn=<SqueezeBackward1>)


In [128]:
x.shape

torch.Size([20, 2])

In [111]:
lstm = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)

In [129]:
print(x)

tensor([[ 0.3437, -0.1859],
        [ 0.0309,  0.7900],
        [ 0.0541,  0.1204],
        [-1.0698, -0.8762],
        [-0.7035,  0.1976],
        [-0.3844,  1.1223],
        [-0.5285,  1.6792],
        [-0.4279, -0.2284],
        [-0.7689,  0.2311],
        [ 1.0001, -0.0284],
        [-0.2504, -0.2195],
        [-0.1710,  0.0692],
        [ 0.2937, -1.0192],
        [-0.7900,  1.1797],
        [ 0.6989, -0.1807],
        [-0.5894,  0.2693],
        [ 0.0839, -0.4948],
        [ 0.4644, -0.7051],
        [ 0.0099,  0.1522],
        [-0.2532,  0.5403]], grad_fn=<SqueezeBackward1>)


In [130]:
h0 = torch.randn(256,2)
c0 = torch.randn(256,2)

In [132]:
o1, _= lstm(x, (h0,c0))

In [133]:
print(o1)

tensor([[ 0.0558, -0.0436],
        [ 0.0992, -0.1022],
        [ 0.1110, -0.1197],
        [ 0.1161, -0.1268],
        [ 0.1188, -0.1298],
        [ 0.1203, -0.1311],
        [ 0.1212, -0.1318],
        [ 0.1218, -0.1321],
        [ 0.1222, -0.1323],
        [ 0.1224, -0.1324],
        [ 0.1226, -0.1325],
        [ 0.1227, -0.1325],
        [ 0.1227, -0.1325],
        [ 0.1228, -0.1325],
        [ 0.1228, -0.1325],
        [ 0.1228, -0.1325],
        [ 0.1228, -0.1325],
        [ 0.1229, -0.1325],
        [ 0.1229, -0.1325],
        [ 0.1229, -0.1325]], grad_fn=<SqueezeBackward1>)


In [134]:
o1.shape

torch.Size([20, 2])

In [118]:
lstm2 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)

In [119]:
h0_2 = torch.randn(256,2)
c0_2 = torch.randn(256,2)

In [135]:
o2, _= lstm(input, (h0_2,c0_2))

In [136]:
o2

tensor([[ 0.0209, -0.3191],
        [ 0.0732, -0.2548],
        [ 0.0938, -0.1598],
        [ 0.1048, -0.1423],
        [ 0.1116, -0.1380],
        [ 0.1158, -0.1361],
        [ 0.1184, -0.1350],
        [ 0.1200, -0.1342],
        [ 0.1210, -0.1337],
        [ 0.1216, -0.1333],
        [ 0.1220, -0.1330],
        [ 0.1223, -0.1329],
        [ 0.1225, -0.1328],
        [ 0.1226, -0.1327],
        [ 0.1227, -0.1326],
        [ 0.1228, -0.1326],
        [ 0.1228, -0.1326],
        [ 0.1228, -0.1326],
        [ 0.1228, -0.1325],
        [ 0.1228, -0.1325]], grad_fn=<SqueezeBackward1>)

In [138]:
lstm3 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)
lstm4 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)


In [139]:
h0_3 = torch.randn(256,2)
c0_3 = torch.randn(256,2)
h0_4 = torch.randn(256,2)
c0_4 = torch.randn(256,2)

In [141]:
o3, _= lstm(input, (h0_3,c0_3))
o4, _= lstm(input, (h0_4,c0_4))

In [142]:
lin = nn.Linear(2,1)

In [143]:
out1 = lin(o1)
out2 = lin(o2)
out3 = lin(o3)
out4 = lin(o4)

In [144]:
out1

tensor([[0.4621],
        [0.4577],
        [0.4567],
        [0.4562],
        [0.4559],
        [0.4556],
        [0.4555],
        [0.4554],
        [0.4553],
        [0.4552],
        [0.4552],
        [0.4552],
        [0.4552],
        [0.4551],
        [0.4551],
        [0.4551],
        [0.4551],
        [0.4551],
        [0.4551],
        [0.4551]], grad_fn=<AddmmBackward0>)

In [145]:
out_comb = torch.stack((out1,out2,out3,out4),dim=1)

In [146]:
out_comb.shape

torch.Size([20, 4, 1])

In [147]:
out_comb = out_comb.squeeze()

In [148]:
out_comb.shape

torch.Size([20, 4])

In [149]:
lin2 = nn.Linear(4,2)

In [150]:
out_final = lin2(out_comb)

In [151]:
out_final

tensor([[-0.3029, -0.1952],
        [-0.3044, -0.1949],
        [-0.2964, -0.1885],
        [-0.2925, -0.1854],
        [-0.2896, -0.1830],
        [-0.2874, -0.1812],
        [-0.2860, -0.1799],
        [-0.2850, -0.1791],
        [-0.2844, -0.1786],
        [-0.2840, -0.1783],
        [-0.2838, -0.1780],
        [-0.2836, -0.1779],
        [-0.2835, -0.1778],
        [-0.2834, -0.1777],
        [-0.2834, -0.1777],
        [-0.2833, -0.1777],
        [-0.2833, -0.1777],
        [-0.2833, -0.1777],
        [-0.2833, -0.1776],
        [-0.2833, -0.1776]], grad_fn=<AddmmBackward0>)